In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as Soup
from random import randint
import pandas as pd
from fake_useragent import UserAgent

In [2]:
driver = webdriver.Edge()
driver.implicitly_wait(20)

In [5]:
current_url = driver.current_url
current_url

'https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=2142800355&p_orderType=6&showType=chessboardType'

In [ ]:
container = pd.DataFrame(columns= ['Products', 'Prices', 'URL', 'Sales'])

In [35]:
prds = driver.find_elements(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[4]/ul/li/a/div[2]/div/h3')
prices = driver.find_elements(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[4]/ul/li/a/div[2]/span[3]')
url = driver.find_elements(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[4]/ul/li/a')
salenum = driver.find_elements(By.XPATH, '//*[@id="goodsAttrRoot"]/div[3]/div[4]/ul/li/a/div[2]/span[4]')
for idx in range(len(prds)):
    container.loc[idx, 'Products'] = prds[idx].get_attribute('textContent')
    container.loc[idx, 'Prices'] = prices[idx].get_attribute('textContent')
    container.loc[idx, 'URL'] = url[idx].get_attribute('href')
    container.loc[idx, 'Sales'] = salenum[idx].get_attribute('textContent')
container

### Process function

In [3]:
def process(number):
  search_key_list = ['HP%20惠普', 'Acer%20宏碁', 'DELL%20戴爾', 'Lenovo']
  brandNo_list = ['20160808155739954', '20160808155608760', '20160808155655766', '20160808155811738']
  search_key = search_key_list[number]
  brandNo = brandNo_list[number]
  driver.get("https://www.momoshop.com.tw/main/Main.jsp")
  brand_url = 'https://www.momoshop.com.tw/search/searchShop.jsp?keyword=' + search_key + '&searchType=1&curPage=1&_isFuzzy=0&brand=' + search_key + '&brandNo='+ brandNo +'&showType=chessboardType&isBrandCategory=N&serviceCode=MT01'
  # 獲取當前網頁的URL
  driver.get(brand_url)
  current_url = driver.current_url

  # 找到"curPage="的位置
  start_index = current_url.find("curPage=")

  if start_index != -1:
      # 找到"curPage="的結尾位置（即數字的結尾位置）
      end_index = current_url.find("&", start_index)
      
      # 提取curPage的數字部分
      cur_page_number = current_url[start_index + len("curPage="):end_index]
      
      # 將curPage的數字替換為{}
      base_url = current_url[:start_index] + "curPage={}" + current_url[end_index:]
      
      # 打印提取的curPage數字
      print("提取的curPage數字是:", cur_page_number)
      
      # 打印修改後的URL
      print("修改後的URL是:", base_url)

  final_page_index = int(driver.find_element(By.XPATH, '//*[@id="BodyBase"]/div[4]/div[6]/dl/dt/span[2]').get_attribute('textContent').split('/')[1])
  print(search_key,"的搜尋結果共有:", final_page_index, '頁')

  Prods_prices = pd.DataFrame(columns=['ProdNames', 'Prices', 'sales'])

  for page_idx in range(1, final_page_index + 1):
      # 根據page_idx生成完整的URL
      url = base_url.format(page_idx)
    
      # 訪問該頁面
      driver.get(url)
      driver.implicitly_wait(randint(5, 10))
      prices = driver.find_elements(By.CLASS_NAME, 'price')
      prdnames = driver.find_elements(By.CLASS_NAME, 'prdName')
      sales = driver.find_elements(By.XPATH, "//span[@class='totalSales goodsTotalSales']")

      for idx in range(len(prices)):
          price_text = prices[idx].text
          prd_text = prdnames[idx].text
          sale_text = sales[idx].get_attribute('textContent')
          new_row = pd.Series({'ProdNames': prd_text, 'Prices': price_text, 'sales':sale_text}).to_frame().T
      
          # 将抓取到的信息添加到 DataFrame
          Prods_prices = pd.concat([Prods_prices, new_row], ignore_index=True)
          
  return Prods_prices

In [56]:
hp = process(0)

提取的curPage數字是: 1
修改後的URL是: https://www.momoshop.com.tw/search/searchShop.jsp?keyword=HP%20%E6%83%A0%E6%99%AE&searchType=1&curPage={}&_isFuzzy=0&brand=HP%20%E6%83%A0%E6%99%AE&brandNo=20160808155739954&showType=chessboardType&isBrandCategory=N&serviceCode=MT04
HP%20惠普 的搜尋結果共有: 36 頁


In [57]:
acer = process(1)

提取的curPage數字是: 1
修改後的URL是: https://www.momoshop.com.tw/search/searchShop.jsp?keyword=Acer%20%E5%AE%8F%E7%A2%81&searchType=1&curPage={}&_isFuzzy=0&brand=Acer%20%E5%AE%8F%E7%A2%81&brandNo=20160808155608760&showType=chessboardType&isBrandCategory=N&serviceCode=MT04
Acer%20宏碁 的搜尋結果共有: 27 頁


In [58]:
dell = process(2)

提取的curPage數字是: 1
修改後的URL是: https://www.momoshop.com.tw/search/searchShop.jsp?keyword=DELL%20%E6%88%B4%E7%88%BE&searchType=1&curPage={}&_isFuzzy=0&brand=DELL%20%E6%88%B4%E7%88%BE&brandNo=20160808155655766&showType=chessboardType&isBrandCategory=N&serviceCode=MT04
DELL%20戴爾 的搜尋結果共有: 9 頁


In [59]:
lenovo = process(3)

提取的curPage數字是: 1
修改後的URL是: https://www.momoshop.com.tw/search/searchShop.jsp?keyword=Lenovo&searchType=1&curPage={}&_isFuzzy=0&brand=Lenovo&brandNo=20160808155811738&showType=chessboardType&isBrandCategory=N&serviceCode=MT04
Lenovo 的搜尋結果共有: 19 頁


In [63]:
acer

,ProdNames,Prices,sales
0,【Acer 宏碁】14吋N5100輕薄筆電(Swift 1/N5100/8G/256G SS...,"$10,888",總銷量>500
1,【Acer 宏碁】14吋i5 12代輕薄筆電(Aspire 5/i5-1235U/8G/51...,"$16,900",總銷量>100
2,【Acer 宏碁】15.6吋i5 13代輕薄筆電 (Aspire 5 /i5-1335U/8...,"$17,900",總銷量>100
3,【Acer 宏碁】15.6吋i3 13代輕薄筆電(Aspire 5/i3-1305U/8G/...,"$13,900",總銷量>100
4,【Acer 宏碁】14吋i5輕薄筆電(Swift 3/EVO/i5-1240P/16G/51...,"$24,900",總銷量>100
...,...,...,...
804,【Acer 宏碁】彩色雷射印表機組★15.6吋R7商用筆電*3(TravelMate TMP...,"$96,999",總銷量>100
805,【Acer 宏碁】附1TB硬碟+8G記憶體組★i5十核電腦(Aspire XC-1780/i...,"$23,900","總銷量>5,000"
806,【Acer 宏碁】彩色雷射印表機組★15.6吋R7商用筆電*3(TravelMate TMP...,"$88,888",總銷量>100
807,【Acer 宏碁】黑白雷射印表機組★15.6吋R7商用筆電*(TravelMate TMP2...,"$82,888",總銷量>100
